# 1. 관광객 추정 버스 이용자 추출 알고리즘

In [1]:
import os
import pandas as pd
from tqdm import tqdm

# 컬럼 상수
user_id = 'user_id'
base_date = 'base_date'
geton_datetime = 'geton_datetime'
geton_station_name = 'geton_stataion_name'
geton_station_longitude = 'geton_station_longitude'
geton_station_latitude = 'geton_station_latitude'
getoff_datetime = 'getoff_datetime'
getoff_station_name = 'getoff_station_name'
getoff_station_longitude = 'getoff_station_longitude'
getoff_station_latitude = 'getoff_station_latitude'
user_count = 'user_count' 

# 자료형 상수
datetime64 = 'datetime64'

# 관광객 연속 체류 기간 (default: 15)
day = 15

### 1) 데이터 불러오기

In [2]:
path = 'c:/tb_bus_user_usage'
if not os.path.isdir(path):
    print("path doesn't exist")
    os.sys.exit(-1)

fname_list = os.listdir(path)
usecols = [user_id, base_date, geton_datetime, geton_station_name, geton_station_longitude, geton_station_latitude,
           getoff_datetime, getoff_station_name, getoff_station_longitude, getoff_station_latitude]

M = pd.read_csv(path + '/' + fname_list[0], usecols=usecols, low_memory=False, encoding = "cp949") #, dtype=dtype)
for fname in tqdm(fname_list[1:]):
    temp_df = pd.read_csv(path + '/' + fname, usecols=usecols, low_memory=False, encoding = "cp949") #, dtype=dtype)
    M = pd.concat([M, temp_df], sort=False, ignore_index=True)

100%|██████████████████████████████████████████████████████████████████████████████████| 89/89 [02:35<00:00,  1.74s/it]


### 2) 데이터 전처리

In [3]:
# datetime64로 형 변환 # M[base_date] = pd.to_datetime(M[base_date], format='%Y%m%d')
datetime_cols = [geton_datetime, getoff_datetime]
for col in tqdm(datetime_cols):
    M[col] = pd.to_datetime(M[col], format='%Y%m%d%H%M%S')
    
print('추출 전 user_id의 개수', len(M[user_id]))

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:49<00:00, 24.93s/it]

추출 전 user_id의 개수 12291789


### 3) 추출1 - 사용기간 고려

In [4]:
'''
관광객으로 유추 가능한 사용자 1차 추출.
1. {user_id, base_date} 쌍을 집합(중복 제거) user_id_and_base_date에 담는다.
    이때, 중복 제거는 {'a1b1c1', '20190101'}, {'a1b1c1', '20190101'} ... 
    => 2회 이상 중복되는 것을 1개만 유지하고 모두 삭제처리한다는 의미.
    * 중복이 생긴 이유는 2019년 1월 1일 a1b1c1이 버스를 3번 탔으면, 3번만큼 튜플이 생기는 것이 당연. 
2. 이후 집합 user_id_and_base_date에 대하여 groupby(user_id).count() 연산을 수행하면,
    각 user가 며칠 버스를 이용하였는지 알 수 있다. (단, 연속적인 며칠 X.) 
    경우1 (1월 1일, 2월 3일, ..., 12월 1일 => 10일): 불연속적인 10일
    경우2 (1월 1일, 2일, 3일, ..., 10일 => 10일): 연속적인 10일
참고로, 2차 추출 단계에서 경우2에 해당하는 튜플만 다시 추출할 것이다.
'''
user_id_and_base_date = M[[user_id, base_date]].drop_duplicates()
date_cnt = user_id_and_base_date.groupby(by=[user_id], as_index=False).count()
print(date_cnt)
U = list(date_cnt[date_cnt[base_date] < day][user_id])
print('1차 추출 결과 user_id의 개수:', len(U))

                                                  user_id  base_date
0       00001a7b32d3addb698c3cfa645cc012ec610aab56d605...          4
1       00003d76b12160017019d1843fe1a15a777c74a986b4ac...          1
2       00003ee373581ff17e54d0d6d4ec813e345c543b783e4f...          1
3       00005481d4d522656d0d389d2593d6e98453c13527099c...          7
4       0000667058c791cad5a33a0859f54cc671dba188896a04...          2
...                                                   ...        ...
726317  ffff67f5fde7d77c9eb3d3a7ff3f7eada790eb5b021a14...         10
726318  ffffb2b0efba979a9e0c16b4ea69c9a1d3e3a9913c48e3...          2
726319  ffffc0a3c55e2a3096df30ecf9c37759218816d236c15d...         30
726320  ffffd090286f3b39e04ace9b714a00a0f775a4b075bdd2...          4
726321  ffffe5c3c6c2ef8b741af4742e77b43b7c20e069d2bc2f...          1

[726322 rows x 2 columns]
1차 추출 결과 user_id의 개수: 605906


### 4) 추출2 - 사용기간 고려

In [5]:
'''
관광객으로 유추 가능한 사용자 2차 추출.
1. 1차 추출에 의해 식별된 user_id의 인스턴스들을 모아 M2에 저장.
2. 이후 user_id별로 base_date의 max와 min을 추출
3. user_id별로 max - min이 14를 초과하는 경우 삭제 처리
   (연속적으로 며칠 이용했는지 판별 -> 14일 이하인 것만 추출.)
'''

M2 = M.query('{} in {}'.format(user_id, U))[[user_id, base_date]].groupby(by=[user_id], as_index=False)
M2_left = M2.max()
M2_right = M2.min()
suffixes = ['_max', '_min']

M2_2 = pd.merge(M2_left, M2_right, on=user_id, how='inner', suffixes=suffixes)
M2_2
for postfix in suffixes:
    M2_2[base_date + postfix] = pd.to_datetime(M2_2[base_date + postfix], format='%Y%m%d')

M2_2['diff'] = M2_2[base_date + suffixes[0]] - M2_2[base_date + suffixes[1]]

U2 = list(M2_2[M2_2['diff'] < '%d days' % day][user_id].unique()) 
print('2차 추출 결과 user_id의 개수:', len(U2))

2차 추출 결과 user_id의 개수: 435662


### 5) 추출3 - 방문 정류장 고려

In [6]:
'''
관광객으로 유추 가능한 사용자 3차 추출.
1. 승/하차 정류장 중 '제주국제공항'이 존재하는가?
2. 승/하차 정류장 중 '제주연안여객터미널', '국제여객선터미널'등이 존재하는가?
3. 승/하차 정류장 중 '호텔/여관/펜션' 등이 존재하는가?
'''
M3 = M.query('{} in {}'.format(user_id, U2))

query_set = []
for col in [geton_station_name, getoff_station_name]:
    for col2 in ['제주국제공항', '제주연안여객터미널', '국제여객선터미널',
                  '호텔', '여관', '펜션', '민박', '관광', '성산', '해수욕장', '공원', '파크', '랜드', '휴양림', '박물관', '폭포', '미술관', '오름']:
        query = '{0}.str.contains("{1}") '.format(col, col2)
        query_set.append(query)
query = ' or '.join(query_set)

M3 = M3.query(query, engine='python')
U3 = list(M3[user_id].unique())

print('3차 추출 결과 user_id의 개수:', len(U3))

3차 추출 결과 user_id의 개수: 281676


### 6) 사용자 아이디 저장 

In [7]:
### (5) 이하 정리 & 저장 #####################################################################################################
df = pd.DataFrame(date_cnt[user_id])
df["tourist"] = df[user_id].isin(U3).apply(lambda x : int(x))
df.to_csv("user_list.csv", encoding="utf-8", index=False)

#유의사항: 실제 사용자(관광객) 수는 user_id의 개수보다 크거나 같을 것. (사용자 수는 user_count로 파악할 것.)

# 추출된 임의의 관광객 이동 패턴 출력 예시

In [8]:
M.query('{} == "{}"'.format(user_id, 
                           '250557c8a203138628194b0ed3dfe9c711c3d21d09d984916c4c9546134f22be'))

,user_id,base_date,geton_datetime,geton_stataion_name,geton_station_longitude,geton_station_latitude,getoff_datetime,getoff_station_name,getoff_station_longitude,getoff_station_latitude
6334036,250557c8a203138628194b0ed3dfe9c711c3d21d09d984...,20190712,2019-07-12 08:29:40,제주국제공항(구제주방면),126.49345,33.50661,2019-07-12 09:17:45,함덕환승정류장(함덕해수욕장),126.66877,33.54101
6516376,250557c8a203138628194b0ed3dfe9c711c3d21d09d984...,20190714,2019-07-14 09:00:38,함덕리(3구),126.66660,33.54171,2019-07-14 09:48:32,제주국제공항(신제주방면),126.49275,33.50610


In [9]:
'055373583405cd9f943d40b199d52570332f3ac91e2f85b726e67da48284d2a8' in U3

True

In [10]:
M.query('{} == "{}"'.format(user_id, 
                           '055373583405cd9f943d40b199d52570332f3ac91e2f85b726e67da48284d2a8'))

,user_id,base_date,geton_datetime,geton_stataion_name,geton_station_longitude,geton_station_latitude,getoff_datetime,getoff_station_name,getoff_station_longitude,getoff_station_latitude
9939611,055373583405cd9f943d40b199d52570332f3ac91e2f85...,20190807,2019-08-07 17:03:30,시외버스출발지(가상정류소),126.51478,33.49946,NaT,NaN,NaN,NaN
10096314,055373583405cd9f943d40b199d52570332f3ac91e2f85...,20190808,2019-08-08 10:26:54,제주시외버스터미널,126.51480,33.49955,2019-08-08 11:54:05,중앙로터리(동),126.56135,33.25297
10096315,055373583405cd9f943d40b199d52570332f3ac91e2f85...,20190808,2019-08-08 19:44:18,플레이케이팝박물관,126.41415,33.25007,2019-08-08 20:41:52,제주국제공항(종점),126.49319,33.50571
10096316,055373583405cd9f943d40b199d52570332f3ac91e2f85...,20190808,2019-08-08 20:56:32,제주국제공항(구제주방면),126.49345,33.50661,2019-08-08 21:07:17,남서광마을입구,126.52126,33.49968
10096317,055373583405cd9f943d40b199d52570332f3ac91e2f85...,20190808,2019-08-08 14:39:54,뉴경남호텔,126.55825,33.24671,NaT,NaN,NaN,NaN
10236732,055373583405cd9f943d40b199d52570332f3ac91e2f85...,20190809,2019-08-09 20:20:42,함덕환승정류장(함덕해수욕장),126.66914,33.54101,2019-08-09 21:03:30,제주버스터미널(종점),126.51632,33.49919
10236733,055373583405cd9f943d40b199d52570332f3ac91e2f85...,20190809,2019-08-09 11:58:44,시외버스출발지(가상정류소),126.51478,33.49946,2019-08-09 12:32:13,삼양2동,126.58042,33.52118
10236734,055373583405cd9f943d40b199d52570332f3ac91e2f85...,20190809,2019-08-09 16:30:16,삼양초등학교,126.58611,33.52112,2019-08-09 16:48:18,함덕환승정류장(함덕해수욕장),126.66877,33.54101
10389965,055373583405cd9f943d40b199d52570332f3ac91e2f85...,20190810,2019-08-10 11:54:25,남서광마을입구,126.52126,33.49968,2019-08-10 12:55:15,만장굴입구,126.77730,33.54925
10389966,055373583405cd9f943d40b199d52570332f3ac91e2f85...,20190810,2019-08-10 17:25:29,동성동,126.75453,33.55577,NaT,NaN,NaN,NaN


In [11]:
M.query('{} == "{}"'.format(user_id, 
                           'f7be0193a64bfedf4f45998a9fe7eb38dd3875c1b6fd10494887ebb5e470023b')).sort_values(by="geton_datetime")

,user_id,base_date,geton_datetime,geton_stataion_name,geton_station_longitude,geton_station_latitude,getoff_datetime,getoff_station_name,getoff_station_longitude,getoff_station_latitude
12251026,f7be0193a64bfedf4f45998a9fe7eb38dd3875c1b6fd10...,20190828,2019-08-28 10:20:02,제주국제공항(구제주방면),126.49345,33.50661,2019-08-28 10:29:32,서문시장,126.51845,33.51203
12251028,f7be0193a64bfedf4f45998a9fe7eb38dd3875c1b6fd10...,20190828,2019-08-28 11:21:15,관덕정,126.52088,33.51322,2019-08-28 11:32:41,어영마을,126.48947,33.51901
12251027,f7be0193a64bfedf4f45998a9fe7eb38dd3875c1b6fd10...,20190828,2019-08-28 13:53:17,어영마을,126.49008,33.51901,2019-08-28 14:08:36,관덕정,126.52096,33.51293
12251029,f7be0193a64bfedf4f45998a9fe7eb38dd3875c1b6fd10...,20190828,2019-08-28 14:13:08,관덕정,126.52088,33.51322,2019-08-28 14:47:46,연동대림1차아파트,126.48975,33.48028
12251025,f7be0193a64bfedf4f45998a9fe7eb38dd3875c1b6fd10...,20190828,2019-08-28 16:16:05,제주고등학교,126.48392,33.47433,2019-08-28 16:58:22,중앙로(동문시장),126.52457,33.51168


In [12]:
M.query('{} == "{}"'.format(user_id, 
                           '1dfeead3f79a0ace23a489fb654fb11a8d45b3cd44adeb8e8822b47397cf3e11')).sort_values(by="geton_datetime")

,user_id,base_date,geton_datetime,geton_stataion_name,geton_station_longitude,geton_station_latitude,getoff_datetime,getoff_station_name,getoff_station_longitude,getoff_station_latitude
11020414,1dfeead3f79a0ace23a489fb654fb11a8d45b3cd44adeb...,20190814,2019-08-14 15:42:25,롯데시티호텔(600번),126.48856,33.49002,NaT,NaN,NaN,NaN
11109640,1dfeead3f79a0ace23a489fb654fb11a8d45b3cd44adeb...,20190815,2019-08-15 09:51:32,솔동산입구,126.56368,33.24474,NaT,NaN,NaN,NaN
11109641,1dfeead3f79a0ace23a489fb654fb11a8d45b3cd44adeb...,20190815,2019-08-15 10:38:44,약천사,126.45034,33.24300,NaT,NaN,NaN,NaN
11109638,1dfeead3f79a0ace23a489fb654fb11a8d45b3cd44adeb...,20190815,2019-08-15 11:34:30,동물개동산앞,126.43120,33.24012,2019-08-15 11:43:10,중문관광단지,126.41271,33.25220
11109637,1dfeead3f79a0ace23a489fb654fb11a8d45b3cd44adeb...,20190815,2019-08-15 14:39:43,천제연폭포,126.41898,33.25395,2019-08-15 15:03:47,산방산,126.31245,33.23623
11109639,1dfeead3f79a0ace23a489fb654fb11a8d45b3cd44adeb...,20190815,2019-08-15 15:42:44,산방산,126.31248,33.23612,2019-08-15 16:19:08,평생학습관,126.55807,33.24950
11202892,1dfeead3f79a0ace23a489fb654fb11a8d45b3cd44adeb...,20190816,2019-08-16 08:00:49,솔동산입구,126.56369,33.24461,2019-08-16 08:03:07,주공34단지,126.57187,33.25010
11202893,1dfeead3f79a0ace23a489fb654fb11a8d45b3cd44adeb...,20190816,2019-08-16 08:04:10,주공34단지,126.57187,33.25010,2019-08-16 09:27:22,성산항입구,126.92917,33.46849
11202891,1dfeead3f79a0ace23a489fb654fb11a8d45b3cd44adeb...,20190816,2019-08-16 15:28:14,성산항입구,126.92955,33.46820,2019-08-16 16:55:35,동문로터리,126.56835,33.24868
11314535,1dfeead3f79a0ace23a489fb654fb11a8d45b3cd44adeb...,20190819,2019-08-19 10:48:05,뉴경남호텔,126.55825,33.24671,NaT,NaN,NaN,NaN


In [13]:
M.query('{} == "{}"'.format(user_id, 
                           '68e5110b22f2fc3eb5d822e52fb3fc96e85f13ecdd25f4fa4afe6255d7b33d58')).sort_values(by="geton_datetime")

,user_id,base_date,geton_datetime,geton_stataion_name,geton_station_longitude,geton_station_latitude,getoff_datetime,getoff_station_name,getoff_station_longitude,getoff_station_latitude
6886920,68e5110b22f2fc3eb5d822e52fb3fc96e85f13ecdd25f4...,20190716,2019-07-16 08:28:46,제주국제공항(구제주방면),126.49345,33.50661,2019-07-16 09:02:59,아라초등학교,126.54428,33.47653
6886917,68e5110b22f2fc3eb5d822e52fb3fc96e85f13ecdd25f4...,20190716,2019-07-16 09:12:20,아라초등학교,126.54428,33.47653,2019-07-16 09:22:03,산천단,126.55539,33.44786
6886916,68e5110b22f2fc3eb5d822e52fb3fc96e85f13ecdd25f4...,20190716,2019-07-16 11:38:58,산천단,126.55539,33.44786,2019-07-16 11:49:37,명도암입구삼거리,126.64044,33.42757
6886918,68e5110b22f2fc3eb5d822e52fb3fc96e85f13ecdd25f4...,20190716,2019-07-16 15:46:03,붉은오름,126.68525,33.39522,2019-07-16 15:48:14,제주목장,126.67799,33.41741
6886919,68e5110b22f2fc3eb5d822e52fb3fc96e85f13ecdd25f4...,20190716,2019-07-16 17:23:19,제주목장,126.67799,33.41741,2019-07-16 17:46:14,산천단한국폴리텍대학,126.55558,33.44799
7010609,68e5110b22f2fc3eb5d822e52fb3fc96e85f13ecdd25f4...,20190717,2019-07-17 06:40:33,사회복지법인춘강,126.55164,33.45264,2019-07-17 07:00:35,제주버스터미널(종점),126.51632,33.49919
7010607,68e5110b22f2fc3eb5d822e52fb3fc96e85f13ecdd25f4...,20190717,2019-07-17 07:22:12,제주시외버스터미널,126.51479,33.49946,2019-07-17 08:32:15,영실매표소,126.48022,33.34025
7010610,68e5110b22f2fc3eb5d822e52fb3fc96e85f13ecdd25f4...,20190717,2019-07-17 14:38:02,영실매표소,126.48022,33.34025,2019-07-17 15:39:52,제주시외버스터미널(종점),126.51414,33.49993
7010611,68e5110b22f2fc3eb5d822e52fb3fc96e85f13ecdd25f4...,20190717,2019-07-17 15:56:29,제주버스터미널,126.51486,33.49993,2019-07-17 16:08:13,제주중앙여자고등학교(아라방면),126.53864,33.48941
7010612,68e5110b22f2fc3eb5d822e52fb3fc96e85f13ecdd25f4...,20190717,2019-07-17 16:53:32,제주중앙여자고등학교(아라방면),126.53864,33.48941,NaT,NaN,NaN,NaN


In [14]:
M.query('{} == "{}"'.format(user_id, 
                           'a6583c22116492a72c059e41272ef7aacdc29f6e9138ede6cf5142c26d780fb4'))

,user_id,base_date,geton_datetime,geton_stataion_name,geton_station_longitude,geton_station_latitude,getoff_datetime,getoff_station_name,getoff_station_longitude,getoff_station_latitude
2993310,a6583c22116492a72c059e41272ef7aacdc29f6e9138ed...,20190620,2019-06-20 07:08:31,제주국제공항(신제주방면),126.49275,33.50610,NaT,NaN,NaN,NaN
3135069,a6583c22116492a72c059e41272ef7aacdc29f6e9138ed...,20190621,2019-06-21 18:40:26,성산항,126.92994,33.47043,2019-06-21 19:54:01,제주버스터미널(종점),126.51632,33.49919
3135070,a6583c22116492a72c059e41272ef7aacdc29f6e9138ed...,20190621,2019-06-21 11:19:06,제주버스터미널,126.51486,33.49993,2019-06-21 12:24:21,성산항,126.93008,33.47032
3208810,a6583c22116492a72c059e41272ef7aacdc29f6e9138ed...,20190622,2019-06-22 11:27:06,명신마을,126.50303,33.49962,2019-06-22 12:23:41,중문환승정류장(중문우체국),126.42380,33.25180
3348669,a6583c22116492a72c059e41272ef7aacdc29f6e9138ed...,20190623,2019-06-23 13:37:26,제주월드컵경기장,126.50862,33.24948,2019-06-23 14:35:38,제주도청신제주로터리,126.49678,33.49143
3348670,a6583c22116492a72c059e41272ef7aacdc29f6e9138ed...,20190623,2019-06-23 14:50:05,제주도청신제주로터리,126.49678,33.49143,2019-06-23 15:08:05,관덕정,126.52096,33.51293
3432113,a6583c22116492a72c059e41272ef7aacdc29f6e9138ed...,20190624,2019-06-24 12:36:27,상창보건진료소,126.36294,33.27464,2019-06-24 12:40:13,상천리사무소,126.37999,33.30723
3432114,a6583c22116492a72c059e41272ef7aacdc29f6e9138ed...,20190624,2019-06-24 16:12:23,제주국제컨벤션센터중문대포해안주상절리대,126.42472,33.24309,2019-06-24 16:54:46,어리목,126.48531,33.39035
3432115,a6583c22116492a72c059e41272ef7aacdc29f6e9138ed...,20190624,2019-06-24 11:28:03,제주도청신제주로터리,126.49647,33.49110,2019-06-24 12:03:24,상창보건진료소,126.36280,33.27458
3432116,a6583c22116492a72c059e41272ef7aacdc29f6e9138ed...,20190624,2019-06-24 15:38:50,제주국제공항(종점),126.49356,33.50572,2019-06-24 16:04:33,중문관광단지여미지식물원입구,126.41264,33.25170
